In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np


class Dosemetry:
    def __init__(self, patient_id):
        self.patient_id = str(patient_id)
        self.patient_df = pd.DataFrame()
        self.fyraften_df = pd.DataFrame()
        self.filtered_df = pd.DataFrame()
        self.total_hours_worked = None  # Store total hours worked
        self.Total_Leq = None
        self.Lex8 = None


    def load_patient_log(self):
        file_path = f"DosimetriData/{self.patient_id}.txt"

        df = pd.read_csv(file_path, sep=r"\s+", header=None, skiprows=11, engine="python")
        df.columns = ['Datapoint', 'Flags', 'Date', 'Time', 'Leq', 'CPEAK', 'Extra']

        if df['Extra'].isna().all():
            df = df.drop(columns=['Extra'])

        df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
        df['Leq'] = pd.to_numeric(df['Leq'].str.replace(',', '.', regex=False), errors='coerce')
        df = df.dropna(subset=['Datetime', 'Leq'])

        self.patient_df = df
        return df


    def load_fyraften(self):
        df = pd.read_csv("DosimetriData/Fyraften.csv", sep=';')
        df.columns = [col.strip().lower() for col in df.columns]

        if 'patientid' not in df.columns:
            if 'personid' in df.columns:
                df = df.rename(columns={'personid': 'patientid'})

        self.fyraften_df = df
        return df


    def limit_patient_data_by_fyraften(self):
        if self.fyraften_df.empty or self.patient_df.empty:
            self.filtered_df = self.patient_df
            self.total_hours_worked = None
            return self.patient_df

        row = self.fyraften_df[self.fyraften_df['patientid'].astype(str) == self.patient_id]

        if row.empty:
            self.filtered_df = self.patient_df
            self.total_hours_worked = None
            return self.patient_df

        fyraften_str = str(row.iloc[0]['fyraften'])

        if not self.patient_df.empty:
            first_date = self.patient_df.iloc[0]['Date']
            fyraften_datetime = pd.to_datetime(f"{first_date} {fyraften_str}", format='%d.%m.%Y %H.%M', errors='coerce')
        else:
            fyraften_datetime = pd.NaT

        filtered_df = self.patient_df[self.patient_df['Datetime'] < fyraften_datetime]
        self.filtered_df = filtered_df

        if not filtered_df.empty:
            start_time = filtered_df.iloc[0]['Datetime']
            end_time = filtered_df.iloc[-1]['Datetime']
            self.total_hours_worked = (end_time - start_time).total_seconds() / 3600.0
        else:
            self.total_hours_worked = None

        return filtered_df


    def calculate_total_leq(self):
        df = self.filtered_df

        if df.empty or 'Leq' not in df:
            return None

        leq_values = df['Leq'].dropna().values
        if len(leq_values) == 0:
            return None

        total_leq = 10 * np.log10(np.mean(10 ** (leq_values / 10)))
        return round(total_leq, 3)


    def calculate_lex8(self):
        leq_t = self.calculate_total_leq()
        T = self.total_hours_worked

        if leq_t is None or T is None or T <= 0:
            return None

        lex8 = leq_t + 10 * np.log10(T / 8)
        return round(lex8, 3)


    def run_full_analysis(self):
        """
        Runs the full analysis workflow for the patient and returns results.
        Returns:
            dict: {'total_leq': value, 'lex8': value, 'total_hours_worked': value},
        """

        self.load_patient_log()
        self.load_fyraften()
        self.limit_patient_data_by_fyraften()

        self.Total_Leq = self.calculate_total_leq()
        self.Lex8 = self.calculate_lex8()

        if self.total_hours_worked is not None:
            self.total_hours_worked = round(self.total_hours_worked, 3)

In [40]:
# Example usage for patient 129
P_129 = Dosemetry(129)
P_129.run_full_analysis()
total_leq = P_129.Total_Leq
lex8 = P_129.Lex8
hours = P_129.total_hours_worked
print(f"Total Leq: {total_leq:.3f}")
print(f"Lex,8: {lex8:.3f}")
print(f"Total hours worked: {hours:.3f}")

Total Leq: 67.365
Lex,8: 67.948
Total hours worked: 9.149


In [41]:
# Script to run analysis for all patients
patient_ids = [129,131,133,142,188,228,242,254,268,386,444,453,480,495,501,537,541,681,689,690,693,711,712,725,728,735,769,779,782,789,815,820,841,864,924,940,955,961,973]
results = {}
for pid in patient_ids:
    patient = Dosemetry(pid)
    patient.run_full_analysis()
    results[pid] = {
        'total_leq': patient.Total_Leq,
        'lex8': patient.Lex8,
        'total_hours_worked': patient.total_hours_worked
    }
# Print results for all patients
for pid, res in results.items():
    print(f"Patient {pid}: Total Leq = {res['total_leq']:.3f}, Lex,8 = {res['lex8']:.3f}, Hours Worked = {res['total_hours_worked']:.3f}")

Patient 129: Total Leq = 67.365, Lex,8 = 67.948, Hours Worked = 9.149
Patient 131: Total Leq = 66.039, Lex,8 = 65.520, Hours Worked = 7.099
Patient 133: Total Leq = 94.691, Lex,8 = 93.343, Hours Worked = 5.865
Patient 142: Total Leq = 67.535, Lex,8 = 67.579, Hours Worked = 8.082
Patient 188: Total Leq = 72.378, Lex,8 = 72.286, Hours Worked = 7.832
Patient 228: Total Leq = 79.950, Lex,8 = 78.807, Hours Worked = 6.149
Patient 242: Total Leq = 84.393, Lex,8 = 82.143, Hours Worked = 4.765
Patient 254: Total Leq = 68.649, Lex,8 = 69.168, Hours Worked = 9.015
Patient 268: Total Leq = 65.911, Lex,8 = 65.392, Hours Worked = 7.099
Patient 386: Total Leq = 71.566, Lex,8 = 71.334, Hours Worked = 7.585
Patient 444: Total Leq = 74.578, Lex,8 = 74.495, Hours Worked = 7.849
Patient 453: Total Leq = 88.908, Lex,8 = 87.765, Hours Worked = 6.149
Patient 480: Total Leq = 78.160, Lex,8 = 79.742, Hours Worked = 11.515
Patient 495: Total Leq = 75.810, Lex,8 = 75.291, Hours Worked = 7.099
Patient 501: Total 

In [42]:
# Load company info from Noeglefil.csv
company_df = pd.read_csv("DosimetriData/Noeglefil.csv", sep=';')
company_df.columns = [col.strip().lower() for col in company_df.columns]
if 'personid' not in company_df.columns and 'patientid' in company_df.columns:
    company_df = company_df.rename(columns={'patientid': 'personid'})
if 'virksomhedsnavn' not in company_df.columns:
    for col in company_df.columns:
        if 'firma' in col:
            company_df = company_df.rename(columns={col: 'virksomhedsnavn'})

# patient_ids from previous script
patient_ids = [129,131,133,142,188,228,242,254,268,386,444,453,480,495,501,537,541,681,689,690,693,711,712,725,728,735,769,779,782,789,815,820,841,864,924,940,955,961,973]

# Run analysis for all patients and store Lex,8
patient_lex8 = {}
for pid in patient_ids:
    patient = Dosemetry(pid)
    patient.run_full_analysis()
    patient_lex8[pid] = patient.Lex8

# Group patients by company and calculate average Lex,8
company_lex8 = {}
for virksomhed in company_df['virksomhedsnavn'].unique():
    pids = company_df[company_df['virksomhedsnavn'] == virksomhed]['personid'].astype(int).tolist()
    lex8_values = [patient_lex8[pid] for pid in pids if pid in patient_lex8 and patient_lex8[pid] is not None]
    if lex8_values:
        avg_lex8 = sum(lex8_values) / len(lex8_values)
    else:
        avg_lex8 = None
    company_lex8[virksomhed] = avg_lex8

# Print comparison of companies
print("Gennemsnitlig Lex,8 pr. virksomhed:")
for virksomhed, avg_lex8 in company_lex8.items():
    if avg_lex8 is not None:
        print(f"{virksomhed}: {avg_lex8:.3f}")
    else:
        print(f"{virksomhed}: None")

# Find company with highest and lowest average Lex,8
valid_companies = {k: v for k, v in company_lex8.items() if v is not None}
if valid_companies:
    max_company = max(valid_companies, key=valid_companies.get)
    min_company = min(valid_companies, key=valid_companies.get)
    print(f"\nHøjeste gennemsnitlige Lex,8: {max_company} ({valid_companies[max_company]:.3f})")
    print(f"Laveste gennemsnitlige Lex,8: {min_company} ({valid_companies[min_company]:.3f})")

Gennemsnitlig Lex,8 pr. virksomhed:
Gebyrgribbenes Sparekasse: 70.007
Svindel og Humbug Bank ApS: 69.214
Haard Metal: 91.011
Vuggestuen Skrig og Skraal: 80.291
Trykkeriet Klatmaling: 84.820

Højeste gennemsnitlige Lex,8: Haard Metal (91.011)
Laveste gennemsnitlige Lex,8: Svindel og Humbug Bank ApS (69.214)
